In [ ]:
# Styling notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("rise.css", "r").read()
    return HTML(styles)
css_styling()

<div style="font-size:2em; text-align:center; margin-top:30px; margin-bottom:20px">Data Science Academy 7</div>
<hr>
<br>

<div style="font-size:4em; text-align:center; margin-bottom:30px; color:#00746E"><b>SARIMAX</b></div>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # data visualisation
import matplotlib.pyplot as plt # data visualisation
import datetime as dt # working with time data
import plotly.graph_objs as go # plotly graphical object

import warnings  
warnings.filterwarnings('ignore')

In [ ]:
# Load dataset
df = pd.read_csv('../input/demand_store_forecast/train.csv')
buf = df[(df['store'] == 1) & (df['item'] == 1)].copy() # item 1 in store 5
buf = buf.set_index('date')
buf.index = pd.DatetimeIndex(buf.index).to_period('D')
y = buf['sales']
# y_to_train = y.iloc[:(len(y)-365)]
# y_to_test = y.iloc[(len(y)-365):] # last year for testing

## <a href='1'>1. Time Series Forecasting</a>

## <a href='1.1'>1.1 SARIMAX preparation</a>

The plot below shows the sales of Item 1 in the store. 

In [ ]:
f, ax1 = plt.subplots(1,1,figsize=(15,5))
y.plot(ax=ax1)
ax1.set_xlabel("Date")
ax1.set_ylabel("Sales of Item 1")
ax1.set_title ("Store 1")
plt.grid(True)

## Train test Split

In [ ]:
#train_test_split
tr_start,tr_end = '2014-01-01','2017-09-30'
te_start,te_end = '2017-10-01','2017-12-31'
tra = buf['sales'][tr_start:tr_end].dropna()
tes = buf['sales'][te_start:te_end].dropna()

The Augmented Dickey-Fuller test can be used to test for stationarity of our time series. The null hypothesis of the test is that the <u>time series is not stationary</u> (has some time-dependent structure).

<font color= '#bfd730'> Null Hypothesis (H0): if failed to be rejected (high p-value) means it is non-stationary

Null Hypothesis (H1): if H0 is rejected (low p-value) means it is stationary</font>

In [ ]:
from statsmodels.tsa.stattools import adfuller

results = adfuller(buf['sales']['2015-01-01':].dropna(), regression = 'ct')
print('ADF Statistic: %f' % results[0])
print('p-value: %f' % results[1])

No surprise. P-value is 0.56 (we don't reject H0) - time series is <u>not stationary</u>. 

To better understand the time series behaviour I will decompose it into <font color='#20419b'>trend, seasonality and residuals.</font>

What is adfuller method parameter 'regression'?

* ’c’ : constant only (default) 
* ’ct’ : constant and trend
* ’ctt’ : constant, and linear and quadratic trend
* ’nc’ : no constant, no trend

## Correlograms
Autocorrelogram & Partial Auto-correlogram is useful that to estimate each models parameters.

In [ ]:
#we use tra.diff()(differenced data), because this time series is unit root process.
fig,ax = plt.subplots(2,1,figsize=(20,10))
fig = sm.graphics.tsa.plot_acf(y.diff().dropna(), lags=50, ax=ax[0])
fig = sm.graphics.tsa.plot_pacf(y.diff().dropna(), lags=50, ax=ax[1])
plt.show()

From the ACF and PACF, we will use the lag number  for ARIMA (p = 2,  d= 1, q= ?). Alternatively, we can use : `arma_order_select_ic` method, it is very easy to search best parameters(p,q) of ARMA model.

In [ ]:
resDiff = sm.tsa.arma_order_select_ic(tra, max_ar=7, max_ma=7, ic='aic', trend='c')
print('ARMA(p,q) =',resDiff['aic_min_order'],'is the best.')

## SARIMA model
I don't know the best way to estimate `seasonal_order(sp,sd,sq,s)`parameters.
parameter s:

* 1 for yearly
* 4 for quarterly
* 12 for monthly
* 52 for weekly
* 365 for daily

In [ ]:
# For now,we choose period 1.
sarima = sm.tsa.statespace.SARIMAX(tra,order=(7,1,7),seasonal_order=(1,1,1,12),
                                enforce_stationarity=False, enforce_invertibility=False,freq='D').fit()
sarima.summary()

In [ ]:
res = sarima.resid
fig,ax = plt.subplots(2,1,figsize=(15,8))
fig = sm.graphics.tsa.plot_acf(res, lags=50, ax=ax[0])
fig = sm.graphics.tsa.plot_pacf(res, lags=50, ax=ax[1])
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
pred = sarima.predict(tr_end,te_end)[1:]
print('SARIMA model MSE: {}'.format(round(mean_squared_error(tes,pred),2)))

In [ ]:
pd.DataFrame({'test':tes,'pred':pred}).plot();plt.show()

## SARIMAX 
Added exogenous variable to ARIMA.

### Make features1
Let's try to make some features.

* month
* dayofweek
* sales_shifted_364(1year_shift)
* sales_shifted_728(2year_shift)


In [ ]:
# Sales groupby month
buf.groupby(buf.index.month).sales.mean().plot();plt.show()

In [ ]:
# Sales groupby day of week
buf.groupby(buf.index.weekday).sales.mean().plot();plt.show()

In [ ]:
plt.plot(buf[0:363].sales.dropna().values)
plt.plot(buf[364:727].sales.dropna().values);plt.show()

Feature Creating with Day of week and Month

In [ ]:
buf = df[(df.item==1)&(df.store==1)].copy()#reset buf
buf = buf.set_index('date')
buf.index = pd.DatetimeIndex(buf.index).to_period('D')
#month one hot encoding
buf['month'] = buf.index.month
month_dummies = pd.get_dummies(buf['month'])
month_dummies.columns = ['month-'+ str(m) for m in range(1,13)]
buf = pd.concat([buf, month_dummies], axis=1).drop(['month'],axis=1)
#dayofweek one hot encoding
buf['dayofweek'] = buf.index.weekday
week_dummies = pd.get_dummies(buf['dayofweek'])
week_dummies.columns = ['dayofweek-'+ str(w) for w in range(0,7)]
buf = pd.concat([buf, week_dummies], axis=1).drop(['dayofweek'],axis=1)
#Satday,Sunday
buf['weekend'] = (buf.index.dayofweek>4).astype(int)#Saturday,Sunday
#Sunday
#buf['sunday'] = (buf.index.dayofweek==6).astype(int)#Saturday,Sunday

In [ ]:
buf.columns

In [ ]:
#shifted data
#buf['sales_shifted_91'] = buf.sales.shift(91)
buf['sales_shifted_728'] = buf.sales.shift(728)
buf['sales_shifted_364'] = buf.sales.shift(364)

In [ ]:
buf[['sales_shifted_728','sales_shifted_364']].head(-200)

In [ ]:
buf[['sales_shifted_728','sales_shifted_364']].tail()

### Split Train Test for ARIMAX model

In [ ]:
tr_start,tr_end = '2015-01-01','2017-09-30'
te_start,te_end = '2017-10-01','2017-12-31'
tra = buf['sales'][tr_start:tr_end].dropna()
tes = buf['sales'][te_start:te_end].dropna()
exog_train = buf.drop(['store','item','sales'],axis = 1)[tr_start:tr_end].dropna()
exog_test = buf.drop(['store','item','sales'],axis = 1)[te_start:te_end].dropna()

In [ ]:
exog_train.head()

## ARIMAX Model

In [ ]:
arimax = sm.tsa.statespace.SARIMAX(tra,order=(7,1,7),seasonal_order=(0,0,0,0),exog = exog_train,freq='D',
                                  enforce_stationarity=False, enforce_invertibility=False,).fit()
arimax.summary()
#We can use SARIMAX model as ARIMAX when seasonal_order is (0,0,0,0) .

In [ ]:
res = arimax.resid
fig,ax = plt.subplots(2,1,figsize=(15,8))
fig = sm.graphics.tsa.plot_acf(res, lags=50, ax=ax[0])
fig = sm.graphics.tsa.plot_pacf(res, lags=50, ax=ax[1])
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
pred = arimax.predict(tr_end,te_end,exog = exog_test)[1:]
print('ARIMAX model MSE:{}'.format(round(mean_squared_error(tes,pred),3)))

In [ ]:
# Visualise the ARIMA-x forecasts actual vs predicted. 
pd.DataFrame({'test':tes,'pred':pred}).plot();plt.show()

In [ ]:
# Visualisation of Residuals Analysis of ARIMAX model
arimax.plot_diagnostics(figsize=(15, 12))

## SARIMAX Model

In [ ]:
sarimax = sm.tsa.statespace.SARIMAX(tra,order=(7,1,7),seasonal_order=(1,1,1,12),exog = exog_train,
                                enforce_stationarity=False, enforce_invertibility=False,freq='D').fit()
sarimax.summary()

In [ ]:
res = sarimax.resid
fig,ax = plt.subplots(2,1,figsize=(15,8))
fig = sm.graphics.tsa.plot_acf(res, lags=50, ax=ax[0])
fig = sm.graphics.tsa.plot_pacf(res, lags=50, ax=ax[1])
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
pred = sarimax.predict(tr_end,te_end,exog = exog_test)[1:]
print('SARIMAX model MSE:{}'.format(mean_squared_error(tes,pred)))

In [ ]:
pd.DataFrame({'test':tes,'pred':pred}).plot();plt.show()

In [ ]:
sarimax.plot_diagnostics(figsize=(15, 12))

It seems that ARIMAX model's prediction is better than SARIMAX model's.
And because SARIMA(X) model has a issue(seasonal period parameter),we choose ARIMAX model.

## ARIMAX Model's summary check

The results of Jarque-Bera test and Ljung-Box test provide an indication of the validity of this model.

In this model's summary, Jarque-Bera test's Prob is under 0.05.
It means that this model's resid is not following a normal distribution.
In other words, some infomations still remain in this model's resid.

Look at the histgram which was output by plot_diagnostics method, It looks like slightly skew.

Ljung-Box test: https://en.wikipedia.org/wiki/Ljung%E2%80%93Box_test

Jarque-Bera test: https://en.wikipedia.org/wiki/Jarque%E2%80%93Bera_test

In [ ]:
arimax.resid.plot();plt.show()
#It seems that there is outlier in this model's resid on late June.

In [ ]:
res_df = pd.DataFrame(arimax.resid,columns=['resid'])
res_df.sort_values(by='resid',ascending=False).head(5)

The outlier is the sales in '2017-06-28'.
Is the date an anniversary or something?

In [ ]:
plt.figure(figsize=(10,15))
piv_val = buf.pivot_table(values='sales',
                          index=buf.index.day,
                          columns=buf.index.month,
                          aggfunc='mean')
sns.heatmap(piv_val)
plt.show()

In [ ]:
buf[(buf.index.day == 28)&(buf.index.month == 6)]['sales']

28th June 2017's sales is too big as other 28th June sales!
Besides, that one day is a weekday.

In [ ]:
#train data predict
pred = arimax.predict(tr_start,tr_end,exog = exog_train)[1:]
pd.DataFrame({'train':tra['2017-06-20':'2017-06-30'],
              'pred':pred['2017-06-20':'2017-06-30']}).plot();plt.show()

### Make Features 2

In [ ]:
#outlier etc...
buf['outlier_flag']=0
buf.loc[buf.index == '2017-06-28','outlier_flag']=1

In [ ]:
tr_start,tr_end = '2015-01-01','2017-09-30'
te_start,te_end = '2017-10-01','2017-12-31'
tra = buf['sales'][tr_start:tr_end].dropna()
tes = buf['sales'][te_start:te_end].dropna()
exog_train = buf.drop(['store','item','sales'],axis = 1)[tr_start:tr_end].dropna()
exog_test = buf.drop(['store','item','sales'],axis = 1)[te_start:te_end].dropna()

In [ ]:
arimax_2 = sm.tsa.statespace.SARIMAX(tra,order=(7,1,7),seasonal_order=(0,0,0,0),exog = exog_train,
                                enforce_stationarity=False, enforce_invertibility=False,freq='D').fit()
arimax_2.summary()

In [ ]:
res = arimax_2.resid
fig,ax = plt.subplots(2,1,figsize=(15,8))
fig = sm.graphics.tsa.plot_acf(res, lags=50, ax=ax[0])
fig = sm.graphics.tsa.plot_pacf(res, lags=50, ax=ax[1])
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
pred = arimax_2.predict(tr_end,te_end,exog = exog_test)[1:]
print('ARIMAX model MSE:{}'.format(mean_squared_error(tes,pred)))

In [ ]:
pd.DataFrame({'test':tes,'pred':pred}).plot();plt.show()

In [ ]:
arimax_2.plot_diagnostics(figsize=(15, 12))

The histgram looks like still skew, but Jarque-Bera test's Prob is over 0.05.
It means that this model's resid is following a normal distribution.

An added featrue was useless to grow up predict accuracy.
but, we were able to make a better model.

### Search best parameters
We can do grid search on the best parameters for SARIMAX. 

<font color = 'red'>Be careful as this will take some time to run! </font>

In [ ]:
import itertools
from sklearn.metrics import mean_squared_error

p = q = range(7,8)
pdq = list(itertools.product(p, [1], q))
sp = sq = range(1,8)#range(0,1) <- ARIMAX
seasonal_pdq = list(itertools.product(sp, [0,1], sq,[12]))#rlist(itertools.product(sp, [0], sq,[0]))<- ARIMAX

params = []
params_s = []
aics = []
mses = []
cnt = 0
for param in pdq:
    for param_seasonal in seasonal_pdq:

        try:
            mod = sm.tsa.statespace.SARIMAX(tra,
                                            order=param,
                                            exog = exog_train,
                                            seasonal_order=param_seasonal,
                                            freq='D',
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            results = mod.fit()

            pred = results.get_prediction(start = pd.to_datetime(tr_end),
                                      end = pd.to_datetime(te_end),exog=exog_test)

            params.append(param)
            params_s.append(param_seasonal)
            aics.append(results.aic)
            mses.append(mean_squared_error(tes,pred.predicted_mean[1:]))


            #if cnt % 8 == 0:
            print('SARIMAX{}x{} - AIC:{} - MSE:{}'.format(param,
                                                            param_seasonal,
                                                            results.aic,
                                                        mses[-1]))
                #cnt += 1

        except:
            continue

min_ind = aics.index(min(aics))
bestparam = (params[min_ind],params_s[min_ind])
print('best_param_aic:',bestparam,' aic:',min(aics))
min_ind = mses.index(min(mses))
bestparam = (params[min_ind],params_s[min_ind])
print('best_param_mse:',bestparam,' mse:',min(mses))

print('Finish!!')